In [1]:
##Imports
import pandas as pd
import pandas_datareader.data as web
from pandas import DataFrame, Series
import datetime
from datetime import timedelta, date
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [9]:
stock = "F"
sp = "^GSPC"
start = (date.today() - timedelta(weeks=260))
end = date.today()
stockDf = web.DataReader(stock, 'yahoo', start, end)
spDf = web.DataReader(sp, 'yahoo', start, end)
stockDf

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-12-12,13.100000,12.810000,13.080000,12.820000,39596800.0,10.391034
2016-12-13,12.880000,12.750000,12.840000,12.770000,39570500.0,10.350508
2016-12-14,12.790000,12.500000,12.650000,12.530000,42820100.0,10.155979
2016-12-15,12.720000,12.520000,12.540000,12.580000,23774800.0,10.196507
2016-12-16,12.700000,12.560000,12.660000,12.630000,28354900.0,10.237032
...,...,...,...,...,...,...
2021-11-29,20.190001,19.490000,20.070000,19.670000,68778700.0,19.670000
2021-11-30,19.950001,19.030001,19.620001,19.190001,103238400.0,19.190001
2021-12-01,20.469999,19.530001,19.629999,19.580000,125295600.0,19.580000


In [10]:
logic = {'Open': 'first',
         'High': 'max',
         'Low': 'min',
         'Close': 'last',
         'Volume': 'sum'}
output = stockDf.resample('W-FRI').agg(logic)
spOutput = spDf.resample('W-FRI').agg(logic)
output

,Open,High,Low,Close,Volume
Date,,,,,
2016-12-16,13.080000,13.100000,12.500000,12.630000,174117100.0
2016-12-23,12.590000,12.820000,12.360000,12.460000,102223400.0
2016-12-30,12.430000,12.510000,12.080000,12.130000,93567600.0
2017-01-06,12.200000,13.270000,12.130000,12.760000,234093200.0
2017-01-13,12.790000,13.020000,12.350000,12.630000,213551000.0
...,...,...,...,...,...
2021-11-05,17.500000,19.719999,17.070000,19.290001,624621600.0
2021-11-12,19.400000,20.510000,19.059999,19.500000,721665400.0
2021-11-19,19.459999,20.260000,19.139999,19.389999,408230200.0


In [11]:
output['30MA'] = output['Close'].rolling(window=30).mean()
stockDf = output.drop(['Open','High','Low'],axis = 1)
spDf = spOutput.drop(['Open','High','Low'],axis = 1)
stockDf

,Close,Volume,30MA
Date,,,
2016-12-16,12.630000,174117100.0,NaN
2016-12-23,12.460000,102223400.0,NaN
2016-12-30,12.130000,93567600.0,NaN
2017-01-06,12.760000,234093200.0,NaN
2017-01-13,12.630000,213551000.0,NaN
...,...,...,...
2021-11-05,19.290001,624621600.0,14.102000
2021-11-12,19.500000,721665400.0,14.344333
2021-11-19,19.389999,408230200.0,14.583333


In [12]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=stockDf.index, y=stockDf['Close'],name='Price'),secondary_y=True)
fig.add_trace(go.Scatter(x=stockDf.index, y=stockDf['30MA'],name="30MA"),secondary_y=True)
fig.add_trace(go.Bar(x=stockDf.index, y=stockDf['Volume'],name="Volume"),secondary_y=False)
fig.show()

In [18]:
stockDf['Percent'] = stockDf['Close'].pct_change()
stockDf['VolumePerc'] = stockDf['Volume'].pct_change() 
spDf['Percent'] = spDf['Close'].pct_change()
stockDf['RS'] = stockDf['Percent'] - spDf['Percent']
stockDf

,Close,Volume,30MA,Percent,RS,VolumePerc
Date,,,,,,
2016-12-16,12.630000,174117100.0,NaN,NaN,NaN,NaN
2016-12-23,12.460000,102223400.0,NaN,-0.013460,-0.015993,-0.412904
2016-12-30,12.130000,93567600.0,NaN,-0.026485,-0.015459,-0.084675
2017-01-06,12.760000,234093200.0,NaN,0.051937,0.034897,1.501862
2017-01-13,12.630000,213551000.0,NaN,-0.010188,-0.009160,-0.087752
...,...,...,...,...,...,...
2021-11-05,19.290001,624621600.0,14.102000,0.129391,0.109382,0.147634
2021-11-12,19.500000,721665400.0,14.344333,0.010886,0.014011,0.155364
2021-11-19,19.389999,408230200.0,14.583333,-0.005641,-0.008868,-0.434322
